# Compute mean of observation for a specific year

In [1]:
# modules importation
from trends_functions import *
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
# run observations
varss = ['od550aer', 'od550lt1aer', 'od550gt1aer', 'ang4487aer', 'concpm25', 'concpm10', 'concso4', 'scatc550dryaer', 'absc550aer']
regions = ['EUROPE', 'NAMERICA', 'SAMERICA', 'NAFRICA', 'SAFRICA', 'ASIA', 'AUSTRALIA']

In [3]:
OBS_MAP = {}
for var in varss:
    if var in ['concpm10', 'concpm25']:
        print('read EBAS')
        EBAS = read_EBAS()
    else:
        EBAS = None
    # computation parameters
    params = get_params()
    params['kind'] = 'obs'
    params['min_dobs'] = 300
    fill_params(params, var)
    if var in ['scatc550dryaer', 'absc550aer']:
        params['period'] = '2000-2018'
    else:
        params['period'] = '2000-2014'
    params['var'] = var
    
    obs_source = params['source']
    reader = pya.io.ReadUngridded(obs_source)
    obs_data = reader.read(vars_to_retrieve=var)
    
    print('compute trend')
    _, OBS_MAP[var], _ = process_trend(
        obs_data, params,
        plot=False, show_plot=False, save_plot=False, write_json=False, EBAS=EBAS
    )

compute trend
no flags
no flags EUROPE station:  ZvenigoroddsEi_GCWD
no flags ASIA station:  Zhongshan_UnivityDh ASIA station:  DRAGON_Korea_Univ ASIA station:  Lahore
no flags NAMERICA station:  Yumaowknife_AuroraC NAMERICA station:  Calipso_Pricestation:  DRAGON_Deer_Park
no flags SAMERICA station:  Veracruz_MAX_MEXFOP
no flags NAFRICA station:  Zinder_DMNporttutea NAFRICA station:  Djougou
region:  SAFRICA station:  Walvis_Bay_airport
Less than 7 points in selected period
no flags
no flags AUSTRALIA station:  Tinga_Tinganand_QLD
compute trendD station:  ZvenigorodportityaPstation:  Calipso_Ridgely DRAGON_BLDND WORLD station:  DRAGON_Jalan_ChainF WORLD station:  La_Crau WORLD station:  Nicelli_Airport WORLD station:  Tenerife WORLD station:  White_Salmon
no flags
no flags EUROPE station:  ZvenigorodMOW1erCWD EUROPE station:  Ispra
no flags ASIA station:  Zhongshan_UnivityDh ASIA station:  DRAGON_Fukue_2
no flags NAMERICA station:  Yumaowknife_AuroraC NAMERICA station:  Calipso_NUFerr

Retrieving EBAS files for variables
['scatc550dryaer']
Reading files 1-131 of 1308 (ReadEbas) | 10:43:34 (delta = 0 s')
Reading files 131-261 of 1308 (ReadEbas) | 10:45:22 (delta = 108 s')
Reading files 261-391 of 1308 (ReadEbas) | 10:46:46 (delta = 84 s')
Reading files 391-521 of 1308 (ReadEbas) | 10:48:16 (delta = 89 s')
Ignoring column 2 (Pyaerocom EbasColDef
--------------------
name: aerosol_light_scattering_coefficient
unit: 1/Mm
is_var: True
is_flag: False
flag_col: 3
) in EBAS file for reading var 
Pyaerocom Variable
------------------
_var_name_input: scatc550aer
is_3d: False
is_dry: False
var_name: scatc550aer
var_name_alt: None
standard_name: None
units: 1/Mm
default_vert_code: None
wavelength_nm: 550.0
dry_rh_max: 40.0
dimensions: None
minimum: -10.0
maximum: 1000.0
description: Aerosol light scattering coefficient at 550 nm
comments_and_purpose: None
obs_wavelength_tol_nm: 10.0
Plotting settings
......................
scat_xlim (list, 2 items)
list (2 items): ['10.0', 1000

compute trend
excluding module
exclude  Montseny
excluding modulestation:  Vavihillmospheric Research Observatoryt
region:  ASIA station:  Gosanon-do
Less than 7 points in selected period
excluding module
exclude  National Capitol - Central, Washington D.C
exclude  Phoenix
excluding moduleA station:  Wichita Mountainsderness3ain #1ment (GA09)WA99)5)
excluding moduleA station:  Cape San Juan
exclude  Granada
region:  NAFRICA station:  Finokaliaillo
Less than 7 points in selected period
excluding module
No station found in , AUSTRALIA Point
excluding module
exclude  Granada
exclude  Montseny
exclude  National Capitol - Central, Washington D.C
exclude  Phoenix


Retrieving EBAS files for variables
['absc550aer']
Ignoring flagged file CA0420G.20100101000000.20190125102503.filter_absorption_photometer.aerosol_absorption_coefficient.aerosol.1y.1h.CA01L_Magee_AE31_ALT.CA01L_aethalometer.lev2.nas
Reading files 1-88 of 874 (ReadEbas) | 10:57:25 (delta = 0 s')
Reading files 88-175 of 874 (ReadEbas) | 10:57:58 (delta = 32 s')
Ignoring column 2 (Pyaerocom EbasColDef
--------------------
name: aerosol_absorption_coefficient
unit: 1/Mm
is_var: True
is_flag: False
flag_col: 4
) in EBAS file for reading var 
Pyaerocom Variable
------------------
_var_name_input: absc550aer
is_3d: False
is_dry: False
var_name: absc550aer
var_name_alt: None
standard_name: None
units: 1/Mm
default_vert_code: None
wavelength_nm: 550.0
dry_rh_max: 40.0
dimensions: None
minimum: -1.0
maximum: 100.0
description: Aerosol light absorption coefficient at 550 nm
comments_and_purpose: None
obs_wavelength_tol_nm: 10.0
Plotting settings
......................
scat_loglog: True
scat_scal

compute trend
excluding module
exclude  Leipzig-Mitte
exclude  Ústí n.L.-mesto
excluding modulestation:  Waldhoflmospheric Research Observatoryt
region:  ASIA station:  Gual Pahariy
Less than 7 points in selected period
excluding module
excluding moduleA station:  Trinidad Headt Plains E13
excluding moduleA station:  Cape San Juan
exclude  Granada
region:  NAFRICA station:  Finokaliailloa
Less than 7 points in selected period
excluding module
No station found in , AUSTRALIA Point
excluding module
exclude  Granada
exclude  Leipzig-Mitte
exclude  Ústí n.L.-mesto


## put this in a dataframe

In [4]:
#'concpm25', 'concpm10', 'concso4', 'scatc550dryaer', 'absc550aer'

In [5]:
regions.remove('SAFRICA')

In [6]:
thead = ['']+regions
tbody = []

for var in varss:
    period = '2000-2014'
    if var in ['concpm25', 'concpm10', 'scatc550dryaer', 'absc550aer']:
        fmt = '{:2.1f}'
        if var in ['scatc550dryaer', 'absc550aer']:
            period = '2000-2018'
    else:
        fmt = '{:3.2f}'
    print(var)
    fill_params(params, var)
    unit = params['unit']
    if unit == '':
        str_unit = ''
    else:
        str_unit = ' ('+unit+')'
    line = [params['ylabel']+str_unit] + [fmt.format(OBS_MAP[var][region]['trends']['trends'][period]['data'][0]) if region in OBS_MAP[var] and 'trends' in OBS_MAP[var][region]['trends'] and period in OBS_MAP[var][region]['trends']['trends'] else '-' for region in regions]
    tbody.append(line)

    df = pd.DataFrame(tbody, columns=thead)

od550aer
od550lt1aer
od550gt1aer
ang4487aer
concpm25
concpm10
concso4
scatc550dryaer
absc550aer


In [7]:
with pd.option_context("max_colwidth", 1000):
    latable = df.to_latex(index=False, longtable=False, escape=False)
    #replace rule with hline for ACP
    latable = latable.replace('toprule','tophline')
    latable = latable.replace('midrule','middlehline')
    latable = latable.replace('bottomrule','bottomhline')
    print(latable)

\begin{tabular}{lllllll}
\tophline
                    & EUROPE & NAMERICA & SAMERICA & NAFRICA &  ASIA & AUSTRALIA \\
\middlehline
                AOD &   0.17 &     0.10 &     0.15 &    0.26 &  0.35 &      0.10 \\
            AOD<1µm &   0.14 &     0.08 &     0.12 &    0.11 &  0.18 &      0.05 \\
            AOD>1µm &   0.03 &     0.02 &     0.03 &    0.10 &  0.11 &      0.03 \\
                 AE &   1.44 &     1.46 &     1.30 &    0.72 &  1.06 &      0.97 \\
     PM2.5 (µg.m-3) &   12.8 &      7.3 &        - &       - &     - &         - \\
      PM10 (µg.m-3) &   16.8 &     12.8 &        - &    19.6 &     - &         - \\
       SO4 (µg.m-3) &   2.01 &     1.45 &        - &    2.98 &  1.97 &         - \\
 Scat. Coef. (1/Mm) &   36.9 &     24.2 &        - &       - &     - &         - \\
  Abs. Coef. (1/Mm) &    5.9 &      2.5 &        - &       - &     - &         - \\
\bottomhline
\end{tabular}

